In [7]:
from SimulateDay import add_columns, get_stock_data
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

stock_df = get_stock_data('INTC')
stock_df = add_columns(stock_df)

/Users/eduardobenjamin/Desktop/Repos/MachineLearningProjects/Stock-Market-Analysis+Model/SimulateDay.py:206: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  '10_Day_Return', '20_Day_Return', '50_Day_Return', '200_Day_Return']].idxmax(axis=1)


In [15]:
def plot_candlestick(df, company,window:list = None,SR_window=10):
    # Fileter data to specified stock and date range
    filtered_df = df[df['Symbol'] == company]
    filtered_df = filtered_df[filtered_df['Date'] >= window[0]]
    filtered_df = filtered_df[filtered_df['Date'] <= window[1]]
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])

    df = filtered_df

    # Create the Candlesick Chart
    candlestick = go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
        )
    
    # Create the Volume Bar graph
    volume = go.Bar(
        x=df['Date'],
        y=df['Volume'],
        name='Volume',
        marker_color='blue',
        opacity=0.5,
        yaxis='y2'
    )

    # Create the Support and Resistance Lines
    support = go.Scatter(
        x=df['Date'],
        y=df[f'Support_{SR_window}_Day'],
        mode='lines',
        name=f'Support {SR_window} Day',
        line=dict(color='green', width=1)
    )

    resistance = go.Scatter(
        x=df['Date'],
        y=df[f'Resistance_{SR_window}_Day'],
        mode='lines',
        name=f'Resistance {SR_window} Day',
        line=dict(color='red', width=1)
    )

    # Create the Layout for the Chart (Title, Axis Labels, etc.)
    layout = go.Layout(
        title= f'{company} Candlestick Chart From {window[0]} to {window[1]}',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Price', showgrid=True),
        yaxis2=dict(title='Volume', overlaying='y', side='right', showgrid=False),
        xaxis_rangeslider_visible=False,
        hovermode='x unified', # Compare data points on hover
        plot_bgcolor='white',
        width=1000,
        height=600
    )

    # Layer all the charts together into one figure
    fig = go.Figure(
        data=[candlestick, volume, support, resistance],
        layout=layout
        )
    return fig


def plot_bollinger_bands(df, company, band):
    # Filter data to specified stock and date range
    df = df[df['Symbol'] == company]
    window=['2023-01-01', '2024-10-01']
    df = df[df['Date'] >= window[0]]
    df = df[df['Date'] <= window[1]]
    fig = go.Figure()

    # Create the Close Price Line Chart
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['Close'],
            mode='lines',
            name='Close Price'
        )
    )

    # Create the upper band
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df[f'upper_band_{band}'],
            mode='lines',
            name='Upper Band',
            line=dict(color='red', width=1)
        )
    )

    # Create the lower band
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df[f'lower_band_{band}'],
            mode='lines',
            fill='tonexty',
            fillcolor='rgba(128, 128, 128, 0.2)',
            name='Lower Band',
            line=dict(color='red', width=1)
        )
    )

    # Update layout
    fig.update_layout(
        title=f'Bollinger Bands for {band} Day Moving Average',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend',
        width=1000,
        height=600
    )

    fig.show()


def plot_MACD(df, company, start, stop):
    # Filter the DataFrame for the specified company and date range
    df = df[(df['Symbol'] == company)] 
    df = df[df['Date'] >= start]
    df = df[df['Date'] <= stop]
    
    # Create a Plotly figure with two subplots
    fig = make_subplots(
        rows=2, 
        cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.1, 
        subplot_titles=('Close Price', 'MACD')
        )
    
    # Add Close Price graph
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Close'],
        mode='lines',
        name='Close Price'
    ), row=1, col=1)
    
    # Add MACD line graph
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['MACD'],
        mode='lines',
        name='MACD',
        line=dict(color='red')
    ), row=2, col=1)
    
    # Add Signal line graph
    fig.add_trace(go.Scatter(
        x=df['Date'], y=df['Signal'],
        mode='lines',
        name='Signal',
        line=dict(color='blue')
    ), row=2, col=1)
    
    # Add MACD histogram 
    fig.add_trace(go.Bar(
        x=df['Date'], y=df['MACD_Hist'],
        name='MACD Histogram',
        marker_color=['green' if x > 0 else 'red' for x in df['MACD_Hist']]
    ), row=2, col=1)
    
    # Update layout
    fig.update_layout(
        title=f'MACD for {company}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend',
        width=1000,
        height=600
    )
    
    fig.show()



In [16]:
plot_candlestick(stock_df, 'INTC', window=['2024-06-01', '2024-10-01'],SR_window=20)

In [12]:
plot_bollinger_bands(stock_df, 'INTC', 20)

In [13]:
plot_MACD(stock_df, 'INTC', '2024-01-01', '2024-10-01')
